<a href="https://colab.research.google.com/github/aeleraqi/GoogleNewsScraper/blob/main/GoogleNewsScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

استخراج الأخبار

In [1]:
!pip install feedparser

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.2 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6090 sha256=6a3c9f8e4e935250f423d112c1ad7b8d066d6d02dbb571b4d3c6e2c2d635a658
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [2]:
import feedparser
from datetime import datetime, timedelta
import logging
import pandas as pd
from urllib.parse import quote
import time

# إعداد التسجيل لمتابعة العمليات
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class GoogleNewsDeepScraper:
    def __init__(self, language='en', region='US'):
        self.language = language
        self.region = region

    def get_month_ranges(self, start_date, end_date):
        """تقسيم الفترة الزمنية الكبيرة إلى فترات شهرية صغيرة"""
        months = []
        current = start_date
        while current < end_date:
            next_month = (current.replace(day=1) + timedelta(days=32)).replace(day=1)
            if next_month > end_date:
                next_month = end_date
            months.append((current, next_month))
            current = next_month
        return months

    def scrape_deep(self, query, start_date, end_date):
        all_articles = []
        date_ranges = self.get_month_ranges(start_date, end_date)

        for s_date, e_date in date_ranges:
            s_str = s_date.strftime('%Y-%m-%d')
            e_str = e_date.strftime('%Y-%m-%d')

            full_query = f"{query} after:{s_str} before:{e_str}"
            encoded_query = quote(full_query)

            rss_url = (
                f"https://news.google.com/rss/search?q={encoded_query}"
                f"&hl={self.language}&gl={self.region}"
                f"&ceid={self.region}:{self.language}"
            )

            logging.info(f"جاري جلب: [{query}] من {s_str} إلى {e_str}")

            try:
                feed = feedparser.parse(rss_url)
                for entry in feed.entries:
                    all_articles.append({
                        'Keyword': query,
                        'Title': entry.title,
                        'Link': entry.link,
                        'Published': entry.published,
                        'Source': entry.source.title if hasattr(entry, 'source') else 'Unknown',
                        'Language': self.language,
                        'Region': self.region
                    })
            except Exception as e:
                logging.error(f"خطأ في الجلب: {e}")

            time.sleep(0.6)

        return all_articles


def run_news_scraper():
    print("\n--- إعدادات البحث العميق في أخبار جوجل ---")

    keywords_input = input("أدخل الكلمات المفتاحية (افصل بينها بفاصلة): ")
    keywords = [k.strip() for k in keywords_input.split(',')]

    language = input("أدخل لغة الأخبار (ar / en / fr): ").strip().lower()
    region = input("أدخل الدولة/المنطقة (EG / US / GB): ").strip().upper()

    start_str = input("تاريخ البداية (YYYY-MM-DD): ")
    end_str = input("تاريخ النهاية (YYYY-MM-DD): ")

    try:
        start_dt = datetime.strptime(start_str, '%Y-%m-%d')
        end_dt = datetime.strptime(end_str, '%Y-%m-%d')
    except ValueError:
        print("❌ خطأ في تنسيق التاريخ")
        return pd.DataFrame()

    scraper = GoogleNewsDeepScraper(language=language, region=region)
    final_list = []

    for kw in keywords:
        results = scraper.scrape_deep(kw, start_dt, end_dt)
        final_list.extend(results)
        print(f"✅ تم جمع {len(results)} خبر مبدئي للكلمة: {kw}")

    if not final_list:
        print("❌ لم يتم العثور على بيانات")
        return pd.DataFrame()

    df = pd.DataFrame(final_list)

    total_before = len(df)
    df = df.drop_duplicates(subset=['Title'])
    total_after = len(df)

    filename = f"news_archive_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df.to_csv(filename, index=False, encoding='utf-8-sig')

    print("\n" + "="*40)
    print(f"إجمالي الأخبار المجمعة: {total_before}")
    print(f"عدد الأخبار الفريدة: {total_after}")
    print(f"تم حذف {total_before - total_after} خبر مكرر")
    print(f"تم حفظ الملف: {filename}")
    print("="*40)

    return df


# تشغيل الكود
df = run_news_scraper()



--- إعدادات البحث العميق في أخبار جوجل ---
أدخل الكلمات المفتاحية (افصل بينها بفاصلة): donald trump
أدخل لغة الأخبار (ar / en / fr): en
أدخل الدولة/المنطقة (EG / US / GB): US
تاريخ البداية (YYYY-MM-DD): 2025-01-01
تاريخ النهاية (YYYY-MM-DD): 2025-12-31
✅ تم جمع 1200 خبر مبدئي للكلمة: donald trump

إجمالي الأخبار المجمعة: 1200
عدد الأخبار الفريدة: 1177
تم حذف 23 خبر مكرر
تم حفظ الملف: news_archive_20260204_092914.csv


In [4]:
import pandas as pd
df = pd.read_csv("/content/news_archive_20260204_092914.csv")

In [ ]:
# 1. إحصائيات الكلمات المفتاحية (Keyword)
print("--- توزيع المقالات حسب الكلمة المفتاحية ---")
print(df['Keyword'].value_counts())
print("-" * 30)

# 2. إحصائيات المصادر (Source) - أعلى 10 مصادر
print("\n--- أعلى 10 مصادر إخبارية تكراراً ---")
print(df['Source'].value_counts().head(10))
print("-" * 30)

# 3. تحليل تاريخ النشر (Published)
# أولاً: تحويل العمود إلى صيغة تاريخ حقيقية لأنها تأتي كنص من RSS
df['Published_DT'] = pd.to_datetime(df['Published'], errors='coerce', utc=True)

# ثانياً: استخراج السنة والشهر للتحليل الزمني
df['Year_Month'] = df['Published_DT'].dt.to_period('M')

print("\n--- توزيع الأخبار حسب الشهر (Timeline) ---")
print(df['Year_Month'].value_counts().sort_index())

--- توزيع المقالات حسب الكلمة المفتاحية ---
Keyword
Egypt politics               1008
Egypt economy                 982
Egypt security                962
Egyptian presidency           774
Egyptian military             745
human rights Egypt            503
Egyptian parliament           469
Egyptian pound                318
Sisi government               315
counterterrorism Egypt        308
inflation Egypt               286
political prisoners Egypt     283
press freedom Egypt           277
IMF Egypt                     254
Sinai insurgency               13
Name: count, dtype: int64
------------------------------

--- أعلى 10 مصادر إخبارية تكراراً ---
Source
Ahram Online           604
Dailynewsegypt         561
Egypt Today            338
Egypt Independent      175
Middle East Monitor    147
The New Arab           126
Reuters                121
The Jerusalem Post     111
Arab News              109
Middle East Eye        100
Name: count, dtype: int64
------------------------------

--- توز

/tmp/ipython-input-1597043222.py:16: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Year_Month'] = df['Published_DT'].dt.to_period('M')


In [ ]:
df

,Keyword,Title,Link,Published,Source,Language,Region,Published_DT,Year_Month
0,Egypt politics,Egypt: Repression intensifies ahead of human r...,https://news.google.com/rss/articles/CBMisgFBV...,"Mon, 27 Jan 2025 08:00:00 GMT",Amnesty International,en,US,2025-01-27 08:00:00+00:00,2025-01
1,Egypt politics,Egypt Remains Cool to the New Syrian Governmen...,https://news.google.com/rss/articles/CBMiigFBV...,"Thu, 23 Jan 2025 08:00:00 GMT",Arab Center Washington DC,en,US,2025-01-23 08:00:00+00:00,2025-01
2,Egypt politics,World Report 2025: Rights Trends in Egypt - Hu...,https://news.google.com/rss/articles/CBMibEFVX...,"Thu, 16 Jan 2025 14:05:39 GMT",Human Rights Watch,en,US,2025-01-16 14:05:39+00:00,2025-01
3,Egypt politics,Assad’s Fall Sparks Fear and Reflection in Egy...,https://news.google.com/rss/articles/CBMiZkFVX...,"Tue, 07 Jan 2025 08:00:00 GMT",New Lines Magazine,en,US,2025-01-07 08:00:00+00:00,2025-01
4,Egypt politics,Reconsidering Cairo's Approach to the Sudanese...,https://news.google.com/rss/articles/CBMiogFBV...,"Thu, 30 Jan 2025 08:00:00 GMT",The Washington Institute,en,US,2025-01-30 08:00:00+00:00,2025-01
...,...,...,...,...,...,...,...,...,...
10038,press freedom Egypt,Salah to the rescue after Egypt start slow - P...,https://news.google.com/rss/articles/CBMid0FVX...,"Tue, 23 Dec 2025 06:45:13 GMT",PressReader,en,US,2025-12-23 06:45:13+00:00,2025-12
10039,press freedom Egypt,Israel approves closing of military radio - Wa...,https://news.google.com/rss/articles/CBMirwFBV...,"Mon, 22 Dec 2025 13:55:04 GMT",Ahram Online,en,US,2025-12-22 13:55:04+00:00,2025-12
10040,press freedom Egypt,Nigeria's AFCON 2025 Journey Begins: Super Eag...,https://news.google.com/rss/articles/CBMimAFBV...,"Tue, 23 Dec 2025 15:15:37 GMT",Head Topics,en,US,2025-12-23 15:15:37+00:00,2025-12
10041,press freedom Egypt,Salah focused on AFCON despite Liverpool uncer...,https://news.google.com/rss/articles/CBMilgFBV...,"Sun, 21 Dec 2025 21:09:11 GMT",Blueprint Newspapers,en,US,2025-12-21 21:09:11+00:00,2025-12


استخراج المحتوى

In [ ]:
!pip install newspaper3k

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 94.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 139.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 9.6 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py

In [ ]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 9.1 MB/s eta 0:00:00


In [ ]:
!pip install lxml_html_clean

In [ ]:
import pandas as pd
from newspaper import Article
from tqdm import tqdm
from datetime import datetime
import time

# 1. دالة سحب محسنة مع معالجة أفضل للأخطاء
def scrape_article(url):
    if pd.isna(url) or str(url).strip() == "":
        return {"Content": None, "Parsed_Date": None, "Error": "URL فارغ"}
    try:
        article = Article(url, language='ar')
        article.download()
        article.parse()
        text = article.text.strip()
        return {
            "Content": text if text else None,
            "Parsed_Date": article.publish_date
        }
    except Exception as e:
        return {"Content": None, "Parsed_Date": None, "Error": str(e)}

# 2. البدء في المعالجة
if 'df' in locals() and not df.empty:
    # تأكد من أننا نستخدم عمود 'Link' الصحيح
    links = df['Link'].tolist()
    print(f"🚀 بدء معالجة {len(links)} رابط فعلياً...")

    texts, publish_dates, errors = [], [], []

    # سنقوم بمعالجة أول 10 روابط فقط كاختبار سريع للتأكد من أن السحب يعمل
    # إذا كنت متأكداً، استبدل links[:10] بـ links لمعالجة الكل
    for url in tqdm(links, desc="جاري سحب المحتوى"):
        result = scrape_article(url)
        texts.append(result.get("Content"))
        publish_dates.append(result.get("Parsed_Date"))
        errors.append(result.get("Error"))
        # إضافة تأخير بسيط جداً لمنع الحظر
        # time.sleep(0.1)

    df['Full_Text'] = texts
    df['Article_Publish_Date'] = publish_dates
    df['Scrape_Error'] = errors

    # 3. تنظيف قاطع لكل أعمدة التاريخ من الـ Timezone
    print("🧹 تنظيف شامل لكل تواريخ الـ DataFrame...")
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]) or df[col].dtype == 'object':
            try:
                df[col] = pd.to_datetime(df[col], errors='ignore')
                if hasattr(df[col], 'dt'):
                    df[col] = df[col].dt.tz_localize(None)
            except:
                pass

    # 4. الحفظ النهائي
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'final_data_{timestamp}.xlsx'

    try:
        # إجبار الحفظ وتجاوز أي خطأ في التنسيق بجعل التواريخ نصوصاً إذا لزم الأمر
        df.to_excel(filename, index=False, engine='openpyxl')
        print(f"✅ تم حفظ الملف بنجاح: {filename}")
    except Exception as e:
        print(f"⚠️ فشل الإكسل، جاري حفظ CSV: {e}")
        df.to_csv(filename.replace('.xlsx', '.csv'), index=False, encoding='utf-8-sig')

else:
    print("❌ لا يوجد بيانات في df")

# فحص عدد المقالات التي نجحنا في سحب نصها فعلياً
successful_scrapes = df['Full_Text'].notna().sum()
print(f"📊 إحصائية: تم سحب محتوى {successful_scrapes} مقال من أصل {len(df)}")

🚀 بدء معالجة 17900 رابط فعلياً...


جاري سحب المحتوى:  49%|████▉     | 8774/17900 [11:31:37<16:34:05,  6.54s/it]

In [ ]:
import pandas as pd
from datetime import datetime
import os

def save_and_download(df):
    # 1. تنظيف أخير وشامل لكل أعمدة التاريخ لحل مشكلة الإكسل (Timezone)
    print("🧹 جاري تجهيز التواريخ للتوافق مع الإكسل...")
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]) or df[col].dtype == 'object':
            try:
                # تحويل إلى تاريخ بدون منطقة زمنية
                df[col] = pd.to_datetime(df[col], errors='ignore')
                if hasattr(df[col], 'dt'):
                    df[col] = df[col].dt.tz_localize(None)
            except:
                continue

    # 2. إنشاء اسم ملف فريد يحتوي على الوقت الحالي
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M')
    excel_name = f'News_Data_Final_{timestamp}.xlsx'
    csv_name = f'News_Data_Final_{timestamp}.csv'

    try:
        # 3. محاولة حفظ الملف بصيغة Excel
        df.to_excel(excel_name, index=False, engine='openpyxl')
        print(f"✅ تم حفظ الملف في ذاكرة السحابة باسم: {excel_name}")
        final_file = excel_name
    except Exception as e:
        # 4. حل احتياطي: حفظ بصيغة CSV إذا فشل الإكسل
        print(f"⚠️ فشل حفظ Excel بسبب: {e}. سيتم الحفظ بصيغة CSV.")
        df.to_csv(csv_name, index=False, encoding='utf-8-sig')
        final_file = csv_name

    # 5. كود إضافي لمستخدمي Google Colab لتحميل الملف لجهازك فوراً
    try:
        from google.colab import files
        print(f"📥 جاري بدء تحميل الملف لجهازك الشخصي...")
        files.download(final_file)
    except ImportError:
        # إذا كنت تعمل على جهازك الشخصي (Local VS Code / PyCharm)
        full_path = os.path.abspath(final_file)
        print(f"📂 الملف جاهز الآن في هذا المسار على جهازك:\n{full_path}")

# تشغيل الدالة
save_and_download(df)

**Visuals**

In [ ]:
import plotly.graph_objects as go

# Line chart for articles over time
line_chart = go.Figure()

line_chart.add_trace(go.Scatter(
    x=articles_per_day.index,
    y=articles_per_day.values,
    mode='lines+markers',
    name='Articles'
))

line_chart.update_layout(
    title='Articles Over Time',
    xaxis_title='Date',
    yaxis_title='Number of Articles',
    xaxis=dict(type='category', tickangle=45)
)
line_chart.show()


In [ ]:
import plotly.express as px

# Reset index to include the index as a column
articles_per_source_df = articles_per_source.reset_index()
articles_per_source_df.columns = ['Source', 'Count']

# Bar chart
bar_chart = px.bar(
    articles_per_source_df,
    x='Source',
    y='Count',
    title='Articles Per Source',
    labels={'Source': 'Source', 'Count': 'Number of Articles'}
)
bar_chart.update_layout(xaxis_title="Source", yaxis_title="Number of Articles")
bar_chart.show()


In [ ]:
# Donut chart for article distribution by source
donut_chart = px.pie(
    articles_per_source_df,
    names='Source',
    values='Count',
    title='Article Distribution by Source',
    hole=0.4,
    labels={'Source': 'Source', 'Count': 'Number of Articles'}
)
donut_chart.show()


In [ ]:
from nltk.stem import PorterStemmer

# Initialize the PorterStemmer
stemmer = PorterStemmer()

# Function to clean and stem text
def clean_and_stem_text(text, sources):
    # Remove special characters
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Remove source names
    for source in sources:
        text = re.sub(rf"\b{re.escape(source)}\b", "", text, flags=re.IGNORECASE)
    # Remove stopwords and apply stemming
    text = " ".join([stemmer.stem(word) for word in text.split() if word.lower() not in stop_words])
    return text

# Apply cleaning and stemming function to 'Title' column
df['Stemmed_Title'] = df['Title'].apply(lambda x: clean_and_stem_text(x, source_names))

# Extract single words (unigrams) using CountVectorizer
vectorizer = CountVectorizer(stop_words='english')  # Unigrams by default
X = vectorizer.fit_transform(df['Stemmed_Title'])
unigram_matrix = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Aggregate unigram frequencies
unigram_frequencies = unigram_matrix.sum(axis=0).sort_values(ascending=False).head(10)

# Convert unigram frequencies into a DataFrame for bar chart
unigram_df = unigram_frequencies.reset_index()
unigram_df.columns = ['Word', 'Frequency']

# Bar chart for unigram frequencies
unigram_bar_chart = px.bar(
    unigram_df,
    x='Word',
    y='Frequency',
    title="Top 10 Stemmed Words in Titles (Source Names Removed)",
    labels={'Word': 'Word', 'Frequency': 'Frequency'},
    text='Frequency'
)

# Customize layout for better readability
unigram_bar_chart.update_layout(
    xaxis_title="Word",
    yaxis_title="Frequency",
    xaxis=dict(tickangle=45),  # Rotate x-axis labels for better readability
    title_font_size=20,
    font=dict(size=14),
    bargap=0.2  # Space between bars
)

# Add text labels to bars
unigram_bar_chart.update_traces(
    textposition='outside',
    marker_color='skyblue',
    marker_line_color='blue',
    marker_line_width=1.5
)

unigram_bar_chart.show()


In [ ]:
# Convert bigram frequencies into a DataFrame for bar chart
bigram_df = bigram_frequencies.reset_index()
bigram_df.columns = ['Bigram', 'Frequency']

# Bar chart for bigram frequencies
bigram_bar_chart = px.bar(
    bigram_df,
    x='Bigram',
    y='Frequency',
    title="Top 10 Bigrams in Titles (Source Names Removed)",
    labels={'Bigram': 'Bigram', 'Frequency': 'Frequency'},
    text='Frequency'
)

# Customize layout for better readability
bigram_bar_chart.update_layout(
    xaxis_title="Bigram",
    yaxis_title="Frequency",
    xaxis=dict(tickangle=45),  # Rotate x-axis labels for better readability
    title_font_size=20,
    font=dict(size=14),
    bargap=0.2  # Space between bars
)

# Add text labels to bars
bigram_bar_chart.update_traces(
    textposition='outside',
    marker_color='skyblue',
    marker_line_color='blue',
    marker_line_width=1.5
)

bigram_bar_chart.show()


In [ ]:
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import nltk
from networkx.algorithms.community import greedy_modularity_communities

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')

# Load English stopwords
stop_words = set(stopwords.words('english'))

# Tokenize, remove stopwords, and count keyword occurrences
keywords = [[word for word in title.split() if word.lower() not in stop_words] for title in df['Title']]
all_keywords = [word for words in keywords for word in words]
keyword_counts = Counter(all_keywords)

# Filter keywords with minimum frequency
min_frequency = 2
filtered_keywords = {word for word, count in keyword_counts.items() if count >= min_frequency}

# Build filtered keyword pairs
keyword_pairs = [
    (word1, word2)
    for words in keywords
    for i, word1 in enumerate(words)
    for word2 in words[i + 1:]
    if word1 in filtered_keywords and word2 in filtered_keywords
]

# Create graph
G = nx.Graph()
G.add_edges_from(keyword_pairs)

# Remove isolated nodes
G.remove_nodes_from(list(nx.isolates(G)))

# Apply community detection (clustering)
communities = greedy_modularity_communities(G)

# Assign cluster labels to nodes
node_colors = {}
for idx, community in enumerate(communities):
    for node in community:
        node_colors[node] = idx

# Map node colors
colors = [node_colors[node] for node in G.nodes]

# Draw graph with clusters
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, k=0.3, iterations=50)
nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=500,
    node_color=colors,
    cmap=plt.cm.tab20,
    font_size=10,
    edge_color="gray"
)
plt.title('Clustered Keyword Co-occurrence Network')
plt.show()


In [ ]:
import feedparser
from datetime import datetime, timedelta
import logging
import pandas as pd
from urllib.parse import quote
import time  # For adding delays

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define the Google News Feed Scraper class
class GoogleNewsFeedScraper:
    def __init__(self, query, start_date, end_date, language, region="US"):
        self.query = query
        self.start_date = start_date  # Already a datetime object
        self.end_date = end_date      # Already a datetime object
        self.language = language
        self.region = region

    def scrape_google_news_feed(self):
        articles = []
        current_date = self.start_date

        while current_date <= self.end_date:
            # Create a time-bound filter by adding a range to the query
            date_filter = f"after:{current_date.strftime('%Y-%m-%d')} before:{(current_date + timedelta(days=1)).strftime('%Y-%m-%d')}"
            query_with_date = f"{self.query} {date_filter}"

            # Encode the query and construct the RSS URL
            encoded_query = quote(query_with_date)
            rss_url = f'https://news.google.com/rss/search?q={encoded_query}&hl={self.language}&gl={self.region}&ceid={self.region}:{self.language[:2]}'

            # Log the constructed RSS URL
            logging.info(f"Fetching RSS feed: {rss_url}")

            # Parse the feed with a delay to avoid rate-limiting issues
            time.sleep(1)  # Add a delay between requests
            feed = feedparser.parse(rss_url)

            if feed.entries:
                for entry in feed.entries:
                    try:
                        # Try to parse the published date from the entry
                        pubdate = datetime.strptime(entry.published, '%a, %d %b %Y %H:%M:%S %Z')
                    except (AttributeError, ValueError):
                        logging.warning(f"Failed to parse date for article: {entry.title}")
                        continue

                    # Check if the article's publication date falls within the specified range
                    if self.start_date <= pubdate <= self.end_date:
                        title = entry.title
                        link = entry.link
                        description = entry.summary if hasattr(entry, 'summary') else entry.description
                        source = entry.source.title if hasattr(entry, 'source') and hasattr(entry.source, 'title') else 'Unknown'
                        articles.append({
                            'Title': title,
                            'Link': link,
                            'Description': description,
                            'Published': pubdate,
                            'Source': source
                        })
            else:
                logging.info(f"No articles found for date: {current_date.strftime('%Y-%m-%d')}")

            current_date += timedelta(days=1)

        return articles

# Function to fetch articles for multiple queries
def fetch_articles(queries, start_date, end_date, language, region="US"):
    all_articles = []

    for query in queries:
        logging.info(f"Fetching news for query: {query.strip()}")
        scraper = GoogleNewsFeedScraper(query.strip(), start_date, end_date, language, region)
        articles = scraper.scrape_google_news_feed()
        all_articles.extend(articles)
        logging.info(f"Fetched {len(articles)} articles for query: {query.strip()}")
        logging.info("="*80)

    return all_articles

# User inputs
keywords = input("Enter keywords (comma separated for multiple): ").split(',')
start_date_str = input("Enter start date (YYYY-MM-DD): ")
end_date_str = input("Enter end date (YYYY-MM-DD): ")
language = input("Enter language (e.g., en for English, ar for Arabic): ")
region = input("Enter region code (e.g., US for the United States, EG for Egypt): ")

# Convert the date strings to datetime objects
try:
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
except ValueError as e:
    print(f"Error: {e}. Please ensure dates are in the format YYYY-MM-DD.")
    raise

# Fetch and display the news articles
all_articles = fetch_articles(keywords, start_date, end_date, language, region)

# Convert the list of articles to a DataFrame and remove duplicates
df = pd.DataFrame(all_articles).drop_duplicates(subset=['Title', 'Link'])

# Save to CSV for review
output_file = "google_news_articles.csv"
df.to_csv(output_file, index=False, encoding="utf-8")
print(f"Articles saved to {output_file}")


In [ ]:
df.info()

In [ ]:
import pandas as pd

In [ ]:
df= pd.read_csv('/content/google_news_articles (2).csv')

In [ ]:
# Cleaning 'Description' column by removing HTML tags
df['Description'] = df['Description'].str.replace(r'<.*?>', '', regex=True)

In [ ]:
from googlenewsdecoder import new_decoderv1

# Function to decode Google News URLs
def decode_google_news_url(url):
    try:
        decoded = new_decoderv1(url)
        if decoded.get("status"):
            return decoded["decoded_url"]
        else:
            return f"Error: {decoded['message']}"
    except Exception as e:
        return f"Exception occurred: {e}"

# Apply decoding function to 'Link' column and create a new 'Original Link' column
df['Original Link'] = df['Link'].apply(decode_google_news_url)

In [ ]:
df.to_excel('ksamarket28to1decen.xlsx', index=False)  # Save the dataframe as an Excel file named 'data.xlsx' and exclude the index column